In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [47]:
datadir ='/Users/francesco/Desktop/Thesis/Data/'

# Read the dataframes
df_bed = xr.open_dataset(datadir + 'BedMachineAntarctica-v3.nc')
df_melt = xr.open_dataset(datadir + 'bmelt_bmachine_grid_v1.nc')

#Adding the names of the glaciers, in order to select later the interesting ones
df_names = pd.read_csv(datadir + 'Merged_Integrated_melt_rates.csv')

In [48]:
df_melt

<xarray.Dataset>
Dimensions:   (x: 13333, y: 13333, time: 26)
Coordinates:
  * x         (x) int32 -3333000 -3332500 -3332000 ... 3332000 3332500 3333000
  * y         (y) int32 3333000 3332500 3332000 ... -3332000 -3332500 -3333000
  * time      (time) datetime64[ns] 1992-01-01 1993-01-01 ... 2017-01-01
Data variables:
    mapping   |S1 ...
    melt      (time, y, x) float64 ...
    melt_err  (time, y, x) float64 ...
Attributes:
    Title:          Antarctica Basal Melt Rates between 1992 and 2017
    Author:         Romain Millan
    version:        28-03-2023
    nx:             13333.0
    ny:             13333.0
    Projection:     Polar Stereographic South (71S,0E)
    proj4:          +init=epsg:3031
    xmin:           -3333000
    ymax:           3333000
    spacing:        500
    Data_citation:  Paolo, F., Gardner, A., Greene, C., Nilsson, J., Schodlok...
    Notes:          These data originate from the Paper of Paolo et al., 2023...

In [49]:
#Extracting the variables

thickness = df_bed.thickness #dim -> (x,y)
melt = df_melt.melt #dim -> (t,x,y)
melt_err = df_melt.melt_err #dim -> (t,x,y)

In [50]:
x = df_bed.x
y = df_bed.y
time = df_melt.time

In [51]:
# Plot the bed with the meshgrid

''' 
#Creatinng the mesh grid
coord = x.values
X, Y = np.meshgrid(coord, coord)


plt.figure(figsize=(10,10))
plt.pcolormesh(X, Y, bed, cmap='gist_earth')
plt.colorbar()
plt.show()
'''


" \n#Creatinng the mesh grid\ncoord = x.values\nX, Y = np.meshgrid(coord, coord)\n\n\nplt.figure(figsize=(10,10))\nplt.pcolormesh(X, Y, bed, cmap='gist_earth')\nplt.colorbar()\nplt.show()\n"

In [52]:
#Now I want to select the areas of the region of interest, and there I want to calculate the basal melt
#just for the thickest part of the glacier, i.e. the part of the glacier where the thickness is above the mean value

In [53]:
import glob
import pandas as pd
import os
from shapely.geometry import shape
import csv
import pyproj
import pyproj
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import fiona
import rasterio
import rasterio.transform
import rasterio.mask
from fiona import Feature, Geometry
from shapely.geometry import mapping, shape

In [54]:
#get information from the shp file

path_to_tif = '/Users/francesco/Desktop/Data/GEOTIFFs/'
shapefile_path = '/Users/francesco/Desktop/Thesis/Data/ice_shelf.shp'

ids = []
Names = []
lat_lon_coords = []
regions = []
boundaries = []
areas = []


#Here i just uses the first tif file to get the information, so 1992
file_tif = path_to_tif + 'melt_' + str(1992) + '_warp_ps.tif'
print(file_tif)


shapefile=fiona.open(shapefile_path)
print(shapefile.schema['properties'])


for feature in shapefile:


    id = feature['properties']['id']
    name=feature['properties']['name']
    geometry=feature['geometry']
    region = feature['properties']['regions']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    #thickness = feature['properties']['thickness_']

    bounds=shape(geometry).bounds
    xmin=bounds[0];ymin=bounds[1];xmax=bounds[2];ymax=bounds[3]

    area_temp = np.abs((xmax-xmin)*(ymax-ymin))

    #Taking the info
    ids.append(id)
    Names.append(name)
    lat_lon_coords.append([lat,lon])
    regions.append(region)
    boundaries.append([xmin,xmax,ymin,ymax])
    areas.append(area_temp)


/Users/francesco/Desktop/Data/GEOTIFFs/melt_1992_warp_ps.tif
{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'f

In [55]:
np.shape(ids)

(179,)

In [56]:
#This block of code is to calculate the mean thickness and the mean melt for the thickest part of the glacier.
#Is commented since the output is already saved in the csv file


#The idea is to create a loop that goes through all the glaciers, 34 mins
#and for each glacier it creates a mask, and then it calculates the mean thickness and the mean melt for the thickest part of the glacier

''' 

Years = np.arange(1992, 2018, 1)
thickness_mean = np.zeros(len(ids))
melt_mean = np.zeros((len(ids), len(time)))
melt_mean_err = np.zeros((len(ids), len(time)))

print(np.shape(melt_mean))


for glacier in range(0, len(ids)):
    x_min , x_max, y_min, y_max =  boundaries[glacier]
    mask = (x > x_min) & (x < x_max) & (y > y_min) & (y < y_max)
    print('Glacier number: ', glacier)
    
    # index both thickness and melt using the mask
    thickness_roi = thickness.where(mask, drop=True)
    thickness_mean_roi = np.mean(thickness_roi)
    thickness_mean[glacier] = thickness_mean_roi

    
    for t in range(0, len(time)):
        melt_roi = melt[t].where(mask, drop=True)
        melt_err_roi = melt_err[t].where(mask, drop=True)

        melt_mean_temp = np.nanmean(melt_roi.where(thickness_roi > thickness_mean_roi))
        melt_mean_err_temp = np.nanmean(melt_err_roi.where(thickness_roi > thickness_mean_roi))

        melt_mean[glacier, t] = melt_mean_temp
        melt_mean_err[glacier, t] = melt_mean_err_temp

'''

" \n\nYears = np.arange(1992, 2018, 1)\nthickness_mean = np.zeros(len(ids))\nmelt_mean = np.zeros((len(ids), len(time)))\nmelt_mean_err = np.zeros((len(ids), len(time)))\n\nprint(np.shape(melt_mean))\n\n\nfor glacier in range(0, len(ids)):\n    x_min , x_max, y_min, y_max =  boundaries[glacier]\n    mask = (x > x_min) & (x < x_max) & (y > y_min) & (y < y_max)\n    print('Glacier number: ', glacier)\n    \n    # index both thickness and melt using the mask\n    thickness_roi = thickness.where(mask, drop=True)\n    thickness_mean_roi = np.mean(thickness_roi)\n    thickness_mean[glacier] = thickness_mean_roi\n\n    \n    for t in range(0, len(time)):\n        melt_roi = melt[t].where(mask, drop=True)\n        melt_err_roi = melt_err[t].where(mask, drop=True)\n\n        melt_mean_temp = np.nanmean(melt_roi.where(thickness_roi > thickness_mean_roi))\n        melt_mean_err_temp = np.nanmean(melt_err_roi.where(thickness_roi > thickness_mean_roi))\n\n        melt_mean[glacier, t] = melt_mean_t

In [57]:
#I now want to save the data in a csv file, so that I can use it later. NEW USED AND CURRENT VERSION IS _3 (this one)
''' 
Years = np.arange(1992, 2018, 1)
df_melt_mean_3 = pd.DataFrame(melt_mean, index=ids, columns=Years)
df_melt_mean_err_3 = pd.DataFrame(melt_mean_err, index=ids, columns=Years)

df_melt_mean_3.to_csv('melt_mean_3.csv')
df_melt_mean_err_3.to_csv('melt_mean_err_3.csv')
'''


" \nYears = np.arange(1992, 2018, 1)\ndf_melt_mean_3 = pd.DataFrame(melt_mean, index=ids, columns=Years)\ndf_melt_mean_err_3 = pd.DataFrame(melt_mean_err, index=ids, columns=Years)\n\ndf_melt_mean_3.to_csv('melt_mean_3.csv')\ndf_melt_mean_err_3.to_csv('melt_mean_err_3.csv')\n"

In [58]:
#open the csv file melt_mean_3.csv and melt_mean_err_3.csv 

df_melt_mean_3 = pd.read_csv('melt_mean_3.csv', header=0, index_col=0)
df_melt_mean_err_3 = pd.read_csv('melt_mean_err_3.csv', header=0, index_col=0)

In [59]:
interesting_glaciers = ['Pine_Island', 'Thwaites', 'Crosson', 'Dotson', 'Getz_2', 'Venable', 'Getz', 'Getz_1' ]
interesting_indices = np.where(np.isin(Names, interesting_glaciers))[0] #this is giving me back the index of the glaciers that I want to plot

In [60]:
interesting_indices

array([43, 44, 52, 61, 67, 68, 69, 71])

In [61]:
df_interesting_glaciers = df_melt_mean_3.iloc[interesting_indices]
df_interesting_glaciers_err = df_melt_mean_err_3.iloc[interesting_indices]

In [62]:
from scipy import stats

Years = np.arange(1992, 2018, 1)

lin_coeff = np.zeros((len(df_interesting_glaciers),2))
lin_reg = np.zeros((len(df_interesting_glaciers),len(Years)))
r_2 = np.zeros(len(df_interesting_glaciers))

for glacier in range(0, len(df_interesting_glaciers)):
    lin_coeff[glacier,:] = np.polyfit(Years, df_interesting_glaciers.iloc[glacier], 1)
    lin_reg[glacier,:] = np.polyval(lin_coeff[glacier,:], Years)
    r_2[glacier] = stats.linregress(Years, df_interesting_glaciers.iloc[glacier])[2]**2

In [63]:
from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('Melt_rates_with_error.pdf')

for glacier in range(0, len(df_interesting_glaciers)):
    fig = plt.figure()
    plt.plot(Years, -df_interesting_glaciers.iloc[glacier], label='Melt rate')
    plt.fill_between(Years, -df_interesting_glaciers.iloc[glacier] - df_interesting_glaciers_err.iloc[glacier], -df_interesting_glaciers.iloc[glacier] + df_interesting_glaciers_err.iloc[glacier], alpha=0.5, label='Error')
    plt.plot(Years, -lin_reg[glacier,:], label='Linear regression')
    plt.title(Names[interesting_indices[glacier]] + ', r^2 = ' + str(round(r_2[glacier], 2)))
    plt.xlabel('Years')
    plt.ylabel('Melt rate [m/yr]')
    plt.legend()
    pp.savefig(fig)
    plt.close()

pp.close()

In [64]:
len(df_melt_mean_3)

179

In [65]:
#Graph for all the glaciers, with error bars but without R^2

from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('Melt_rates_with_error_all_glaciers.pdf')

for glacier in range(0, len(df_melt_mean_3)):
    fig = plt.figure()
    plt.plot(Years, -df_melt_mean_3.iloc[glacier], label='Melt rate')
    plt.fill_between(Years, -df_melt_mean_3.iloc[glacier] - df_melt_mean_err_3.iloc[glacier], -df_melt_mean_3.iloc[glacier] + df_melt_mean_err_3.iloc[glacier], alpha=0.5, label='Error')
    plt.title(Names[glacier])
    plt.xlabel('Years')
    plt.ylabel('Melt rate [m/yr]')
    plt.legend()
    pp.savefig(fig)
    plt.close()

pp.close()



In [66]:
#Here i check how many glaciers have have no time series value

x = df_melt_mean_3.isnull().sum(axis=1)
np.shape(x)

#y is the number of glaciers that have no time series value
y =x.sum()/26
y


39.0